# Algorytmy tekstowe 2019/2020

# Laboratorium 2

# Autor - Łukasz Jezapkowicz

# 1. Przyjmij następujący zbiór danych wejściowych:
## - bbb$
## - aabbabd
## - ababcd
## - abcbccd
## - załączony plik ojczyzna.txt (powielona x10 Litwo, Ojczyzno moja!)

Dla ciekawych:
Litwo, Ojczyzno moja! ty jesteś jak zdrowie;
Ile cię trzeba cenić, ten tylko się dowie,
Kto cię stracił. Dziś piękność twą w całej ozdobie
Widzę i opisuję, bo tęsknię po tobie.
Panno święta, co Jasnej bronisz Częstochowy
I w Ostrej świecisz Bramie! Ty, co gród zamkowy
Nowogródzki ochraniasz z jego wiernym ludem!
Jak mnie dziecko do zdrowia powróciłaś cudem
(— Gdy od płaczącej matki, pod Twoją opiekę
Ofiarowany martwą podniosłem powiekę;
I zaraz mogłem pieszo, do Twych świątyń progu
Iść za wrócone życie podziękować Bogu —)
Tak nas powrócisz cudem na Ojczyzny łono!...
Tymczasem, przenoś moją duszę utęsknioną
Do tych pagórków leśnych, do tych łąk zielonych,
Szeroko nad błękitnym Niemnem rozciągnionych;
Do tych pól malowanych zbożem rozmaitem,
Wyzłacanych pszenicą, posrebrzanych żytem;
Gdzie bursztynowy świerzop, gryka jak śnieg biała,
Gdzie panieńskim rumieńcem dzięcielina pała,
A wszystko przepasane jakby wstęgą, miedzą
Zieloną, na niej zrzadka ciche grusze siedzą.

In [27]:
data = []
data.append("bbb$")
data.append("aabbabd")
data.append("ababcd")
data.append("abcbccd")
file = open('ojczyzna.txt',mode='r', encoding="utf-8")
data.append(file.read())
file.close()
data[4] = data[4]+'$' # marker

# 2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak. 

Łatwo zauważyć, że wszystkie dane wejściowe oprócz pliku ojczyzna.txt posiada unikalny znak na końcu (marker). Dopisałem więc na jego końcu znak $. Dzięki temu każdy łańcuch posiada unikalny znak na końcu.

# 3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

In [28]:
# Klasa reprezentująca węzeł w drzewie Trie
class TrieNode:
    def __init__(self):
        self.leaf = False
        self.char = None
        self.parent = None
        self.link = None
        self.children = dict()

In [29]:
# Klasa reprezentująca drzewo Trie
class Trie:
    def __init__(self):
        self.root = TrieNode()
        
    # metoda znajdujaca poczatkowe drzewo (po dodaniu 1 sufiksu)
    def compute_init_tree(self,suffix):
        rt = self.root
        i = 0
        while i < len(suffix):
            rt.children[ord(suffix[i])] = TrieNode()
            rt.children[ord(suffix[i])].parent = rt # ojciec
            rt = rt.children[ord(suffix[i])]
            rt.char = suffix[i]
            if i == 0:
                rt.link = self.root # tworze link od syna roota do roota
            i += 1
        rt.leaf = True # oznaczam ostatni węzeł liściem
        return rt # zwracam liść
    
    # metoda up-link-down
    def up_link_down(self,leaf):
        v = leaf
        txt = ""
        # UP
        while v.parent and v.link is None:
            txt = v.char + txt
            v = v.parent
        if v.link is None:
            return None, None,txt
        # LINK
        head = v.link
        # DOWN
        i = 0
        while ord(txt[i]) in head.children: # póki istnieja odpowiednie dzieci
            v = v.children[ord(txt[i])]
            head = head.children[ord(txt[i])]
            v.link = head
            i += 1
        return v,head,txt[i:]
        
    # metoda dodajaca brakujaca sciezke
    def graft(self,v,head,txt):
        w = head
        for i in range(len(txt)):
            v = v.children[ord(txt[i])]
            if ord(txt[i]) not in w.children:
                w.children[ord(txt[i])] = TrieNode()
                w.children[ord(txt[i])].parent = w
                w.children[ord(txt[i])].char = txt[i]
            w = w.children[ord(txt[i])]
            v.link = w
        w.leaf = True
        return w
    
    # metoda budująca drzewo Trie dla danego tekstu
    def build_trie(self,text):
        leaf = self.compute_init_tree(text) # poczatkowe drzewo
        for i in range(1,len(text)):
            v,head,txt = self.up_link_down(leaf)
            if head is None: # specjalny przypadek
                v = self.root.children[ord(text[i-1])]
                v.link = self.root
                head = self.root
                txt = txt[1:]
            leaf = self.graft(v,head,txt)
        
    
    # metoda odszukujaca czy podsłowo nalezy do drzewa
    def search(self,word):
        rt = self.root
        i = 0
        while i < len(word) and ord(word[i]) in rt.children:
            rt = rt.children[ord(word[i])]
            i += 1
        if i == len(word):
            return True
        return False
    
    def get_all_suffixes(self,node,suffix):
        if node.leaf:
            print(suffix)
            return
        for key in node.children:
            self.get_all_suffixes(node.children[key],suffix+node.children[key].char)

In [30]:
from time import time
# Klasa odpowiedzialna za mierzenie czasów budowania odpowiednich drzew
class TimeMeasure:
    def measure_trie(self,text):
        time1 = time()
        trie = Trie()
        trie.build_trie(text)
        time2 = time()
        if len(text) < 20:
            print("Building trie from text " + text + " took " + str(1000*(time2-time1)) + " milliseconds")
        else:
            print("Building trie from text from file 'ojczyzna.txt' took " + str(time2-time1) + " seconds")

In [31]:
tm = TimeMeasure()
for text in data:
    tm.measure_trie(text)

Building trie from text bbb$ took 0.0 milliseconds
Building trie from text aabbabd took 0.0 milliseconds
Building trie from text ababcd took 0.0 milliseconds
Building trie from text abcbccd took 0.0 milliseconds
Building trie from text from file 'ojczyzna.txt' took 42.137665033340454 seconds


## Pokaz działania - wypisanie wszystkich sufiksów "aabbabd"

In [32]:
x = Trie()
x.build_trie(data[1])
x.get_all_suffixes(x.root,"")

aabbabd
abbabd
abd
bbabd
babd
bd
d


# 4. Zaimplementuj algorytm konstruujący drzewo sufiksów w oparciu o algorytm McCreighta.

## a) bez linków i fast_find

In [33]:
# Klasa reprezentująca węzeł w drzewie sufiksów
class SuffixEasyNode:
    def __init__(self):
        self.leaf = False
        self.l_index = None
        self.r_index = None
        self.parent = None
        self.link = None
        self.children = dict()

In [34]:
# Klasa reprezentująca drzewo Trie
class SuffixEasyTree:
    def __init__(self):
        self.root = SuffixEasyNode()
        self.text = None
        
    # metoda znajdujaca poczatkowe drzewo (po dodaniu 1 sufiksu)
    def compute_init_tree(self,suffix):
        self.text = suffix  # zapisuje caly tekst 
        rt = self.root
        rt.children[ord(suffix[0])] = SuffixEasyNode()
        rt.children[ord(suffix[0])].parent = rt
        rt = rt.children[ord(suffix[0])]
        rt.l_index = 0
        rt.r_index = len(suffix)-1
        rt.leaf = True # oznaczam ostatni węzeł liściem
        return rt # zwracam liść
    
    # metoda break
    def breakNode(self,node1,node2,index):
        new_node = SuffixEasyNode()
        i = node2.l_index
        j = node2.r_index
        new_node.parent = node1
        new_node.l_index = i
        new_node.r_index = i+index-1
        new_node.children[ord(self.text[i+index])] = node2
        node1.children[ord(self.text[i])] = new_node
        node2.l_index += index
        return new_node
        
    
    # metoda slow find
    def slow_find(self,node,txt):
        i = 0
        result = None
        while i < len(txt) and ord(txt[i]) in node.children:
            child = node.children[ord(txt[i])]
            for j in range(child.l_index,child.r_index+1):
                if not txt[i] == self.text[j]:
                    result = self.breakNode(node,child,j-child.l_index)
                    break
                i += 1
            if result is not None:
                break
            node = child
        if result is None:
            result = node
        return result,i
            
    def easy_insert(self,shift,head):
        child = SuffixEasyNode()
        child.parent = head
        child.l_index = shift
        child.r_index = len(self.text)-1
        child.leaf = True
        return child
    
    # metoda budująca drzewo Trie dla danego tekstu bez linkow i fast_find (od roota)
    def build_tree_easy(self,text):
        leaf = self.compute_init_tree(text) # poczatkowe drzewo
        for i in range(1,len(text)):
            suff = text[i:]
            head,shift = self.slow_find(self.root,suff)
            txt = suff[shift:]
            head.children[ord(txt[0])] = self.easy_insert(i+shift,head)

    def get_all_suffixes(self,node,suffix):
        if node.leaf:
            print(suffix)
        for key in node.children:
            child = node.children[key]
            self.get_all_suffixes(child,suffix+self.text[child.l_index:child.r_index+1])

In [35]:
# Klasa odpowiedzialna za mierzenie czasów budowania odpowiednich drzew
class TimeMeasure:
    def measure_trie(self,text):
        time1 = time()
        tree = SuffixEasyTree()
        tree.build_tree_easy(text)
        time2 = time()
        if len(text) < 20:
            print("Building easy suffix tree from text " + text + " took " + str(1000*(time2-time1)) + " milliseconds")
        else:
            print("Building easy suffix tree from text from file 'ojczyzna.txt' took " + str(time2-time1) + " seconds")

In [36]:
tm = TimeMeasure()
for text in data:
    tm.measure_trie(text)

Building easy suffix tree from text bbb$ took 0.0 milliseconds
Building easy suffix tree from text aabbabd took 0.0 milliseconds
Building easy suffix tree from text ababcd took 0.0 milliseconds
Building easy suffix tree from text abcbccd took 0.0 milliseconds
Building easy suffix tree from text from file 'ojczyzna.txt' took 5.396024942398071 seconds


## Pokaz działania - wypisanie wszystkich sufiksów "aabbabd"

In [37]:
x = SuffixEasyTree()
x.build_tree_easy(data[1])
x.get_all_suffixes(x.root,"")

aabbabd
abbabd
abd
bbabd
babd
bd
d


## b) z linkami i fast_find 

In [38]:
# Klasa reprezentująca węzeł w drzewie sufiksów
class SuffixNode:
    def __init__(self):
        self.leaf = False
        self.l_index = None
        self.r_index = None
        self.parent = None
        self.link = None
        self.children = dict()
    
    def __str__(self):
        return "l_index = " + str(self.l_index) + " r_index = " + str(self.r_index) + " children = " + str(self.children)

In [39]:
# Klasa reprezentująca drzewo Trie
class SuffixTree:
    def __init__(self):
        self.root = SuffixNode()
        self.text = None
        
    # metoda znajdujaca poczatkowe drzewo (po dodaniu 1 sufiksu)
    def compute_init_tree(self,suffix):
        self.text = suffix  # zapisuje cały tekst 
        rt = self.root
        rt.children[ord(suffix[0])] = SuffixNode()
        rt.children[ord(suffix[0])].parent = rt.children[ord(suffix[0])].link = rt 
        rt.link = rt.parent = rt # link roota na samego siebie
        rt = rt.children[ord(suffix[0])]
        rt.l_index = 0
        rt.r_index = len(suffix)-1
        rt.leaf = True # oznaczam ostatni węzeł liściem
        return rt # zwracam liść
    
    # metoda break
    def breakNode(self,node1,node2,txt):
        i = 0
        j = node2.l_index
        while i < len(txt) and txt[i] == self.text[j]:
            i += 1
            j += 1
        new_node = SuffixNode()
        new_node.parent = node1
        new_node.l_index = node2.l_index
        new_node.r_index = node2.l_index+i-1
        new_node.link = node1
        new_node.children[ord(self.text[j])] = node2
        node1.children[ord(txt[0])] = new_node
        node2.l_index += i
        node2.parent = new_node
        return new_node, len(txt)-i
    
    # metoda fast_find
    def fast_find(self,node,txt):
        while len(txt) > 0 and ord(txt[0]) in node.children:
            child = node.children[ord(txt[0])]
            length = child.r_index-child.l_index+1
            if len(txt) < length:
                return self.breakNode(node,child,txt)
            txt = txt[length:]
            node = child
        return node,len(txt)
    
    # metoda slow find
    def slow_find(self,node,txt):
        while len(txt) > 0:
            child = node.children[ord(txt[0])]
            i = 0
            j = child.r_index
            length = child.r_index-child.l_index+1
            if len(txt) >= length:
                while i < length:
                    if txt[i] != self.text[j]:
                        return self.breakNode(node,child,txt)
                    i += 1
                    j += 1
            else:
                return self.breakNode(node,child,txt)
            txt = txt[length:]
            node = child
            if len(txt)>0 and ord(txt[0]) not in node.children:
                break
        return node,len(txt)
    
    # metoda up-link-down (zwraca (head,index_poczatku_koncowki))
    def up_link_down(self,leaf,suff):
        head = leaf.parent
        parent = head.parent
        father_head = ""
        if head != parent: # head == parent -> root
            father_head = self.text[head.l_index:head.r_index+1]
        head_leaf = self.text[leaf.l_index:leaf.r_index+1]
        u = parent.link
        v, tail_len = self.fast_find(u,father_head)
        if head == parent or v.parent == u: # specjalny przypadek
            if ord(suff[0]) in u.children:
                new_head, tail_len = self.slow_find(u,suff)
            else:
                new_head, tail_len = u,len(suff)
        else:
            new_head, tail_len = v,len(head_leaf)
        if head != v:
            head.link = v
        return new_head,tail_len
        
    # metoda graft
    def graft(self,head,tail_len):
        child = SuffixNode()
        child.leaf = True
        child.l_index = len(self.text)-tail_len
        child.r_index = len(self.text)-1
        child.parent = head
        head.children[ord(self.text[child.l_index])] = child
        return child
            
    # metoda budująca drzewo Trie dla danego tekstu bez linkow i fast_find (od roota)
    def build_tree(self,text):
        leaf = self.compute_init_tree(text) # poczatkowe drzewo
        for i in range(1,len(text)):
            suff = text[i:]
            head,tail_len = self.up_link_down(leaf,suff)
            leaf = self.graft(head,tail_len)

    def get_all_suffixes(self,node,suffix):
        if node.leaf:
            print(suffix)
        for key in node.children:
            child = node.children[key]
            self.get_all_suffixes(child,suffix+self.text[child.l_index:child.r_index+1])

In [40]:
# Klasa odpowiedzialna za mierzenie czasów budowania odpowiednich drzew
class TimeMeasure:
    def measure_trie(self,text):
        time1 = time()
        tree = SuffixTree()
        tree.build_tree(text)
        time2 = time()
        if len(text) < 20:
            print("Building suffix tree from text " + text + " took " + str(1000*(time2-time1)) + " milliseconds")
        else:
            print("Building suffix tree from text from file 'ojczyzna.txt' took " + str(time2-time1) + " seconds")

In [41]:
tm = TimeMeasure()
for text in data:
    tm.measure_trie(text)

Building suffix tree from text bbb$ took 0.0 milliseconds
Building suffix tree from text aabbabd took 0.0 milliseconds
Building suffix tree from text ababcd took 0.0 milliseconds
Building suffix tree from text abcbccd took 0.0 milliseconds
Building suffix tree from text from file 'ojczyzna.txt' took 7.919973611831665 seconds


## Pokaz działania - wypisanie wszystkich sufiksów "aabbabd"

In [44]:
x = SuffixTree()
x.build_tree(data[1])
x.get_all_suffixes(x.root,"")

aabbabd
abbabd
abd
bbabd
babd
bd
d


# 5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.

# Więcej przykładów

In [51]:
x = Trie()
x.build_trie(data[0])
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixEasyTree()
x.build_tree_easy(data[0])
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixTree()
x.build_tree(data[0])
x.get_all_suffixes(x.root,"")

bbb$
bb$
b$
$
-------------------------
bbb$
bb$
b$
$
-------------------------
bbb$
bb$
b$
$


In [52]:
x = Trie()
x.build_trie(data[2])
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixEasyTree()
x.build_tree_easy(data[2])
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixTree()
x.build_tree(data[2])
x.get_all_suffixes(x.root,"")

ababcd
abcd
babcd
bcd
cd
d
-------------------------
ababcd
abcd
babcd
bcd
cd
d
-------------------------
ababcd
abcd
babcd
bcd
cd
d


In [53]:
x = Trie()
x.build_trie(data[3])
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixEasyTree()
x.build_tree_easy(data[3])
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixTree()
x.build_tree(data[3])
x.get_all_suffixes(x.root,"")

abcbccd
bcbccd
bccd
cbccd
ccd
cd
d
-------------------------
abcbccd
bcbccd
bccd
cbccd
ccd
cd
d
-------------------------
abcbccd
bcbccd
bccd
cbccd
ccd
cd
d


In [56]:
x = Trie()
x.build_trie("Algorytmy Tekstowe są git.")
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixEasyTree()
x.build_tree_easy("Algorytmy Tekstowe są git.")
x.get_all_suffixes(x.root,"")
print("-------------------------")
x = SuffixTree()
x.build_tree("Algorytmy Tekstowe są git.")
x.get_all_suffixes(x.root,"")

Algorytmy Tekstowe są git.
lgorytmy Tekstowe są git.
gorytmy Tekstowe są git.
git.
orytmy Tekstowe są git.
owe są git.
rytmy Tekstowe są git.
ytmy Tekstowe są git.
y Tekstowe są git.
tmy Tekstowe są git.
towe są git.
t.
my Tekstowe są git.
 Tekstowe są git.
 są git.
 git.
Tekstowe są git.
ekstowe są git.
e są git.
kstowe są git.
stowe są git.
są git.
we są git.
ą git.
it.
.
-------------------------
Algorytmy Tekstowe są git.
lgorytmy Tekstowe są git.
gorytmy Tekstowe są git.
git.
orytmy Tekstowe są git.
owe są git.
rytmy Tekstowe są git.
ytmy Tekstowe są git.
y Tekstowe są git.
tmy Tekstowe są git.
towe są git.
t.
my Tekstowe są git.
 Tekstowe są git.
 są git.
 git.
Tekstowe są git.
ekstowe są git.
e są git.
kstowe są git.
stowe są git.
są git.
we są git.
ą git.
it.
.
-------------------------
Algorytmy Tekstowe są git.
lgorytmy Tekstowe są git.
gorytmy Tekstowe są git.
git.
orytmy Tekstowe są git.
owe są git.
rytmy Tekstowe są git.
ytmy Tekstowe są git.
y Tekstowe są git.
tmy Tekstow

# 6. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1:

Building trie from text bbb$ took 0.0 milliseconds  
Building trie from text aabbabd took 0.0 milliseconds  
Building trie from text ababcd took 0.0 milliseconds  
Building trie from text abcbccd took 0.0 milliseconds  
Building trie from text from file 'ojczyzna.txt' took 42.137665033340454 seconds  

Building easy suffix tree from text bbb$ took 0.0 milliseconds  
Building easy suffix tree from text aabbabd took 0.0 milliseconds  
Building easy suffix tree from text ababcd took 0.0 milliseconds  
Building easy suffix tree from text abcbccd took 0.0 milliseconds  
Building easy suffix tree from text from file 'ojczyzna.txt' took 5.396024942398071 seconds  

Building suffix tree from text bbb$ took 0.0 milliseconds  
Building suffix tree from text aabbabd took 0.0 milliseconds  
Building suffix tree from text ababcd took 0.0 milliseconds  
Building suffix tree from text abcbccd took 0.0 milliseconds  
Building suffix tree from text from file 'ojczyzna.txt' took 7.919973611831665 seconds  

#### Jak widać, moja implementacja drzewa sufiksów w wariancie z linkowaniem i fast_findem okazała się nie do końca prawidłowa. Czas tworzenia drzewa okazał się dłuższy niż dla wariantu bez linkowania i fast_finda. O pomyłkę nie było trudno, albowiem algorytm nie jest szczególnie łatwy. Podejrzewam błąd w sposobie działania budowania drzewa dla początkowych sufiksów (nigdzie w książce nie było dobrze opisane jak algorytm zacząć, algorytm był opisany dla przypadków, gdzie faktycznie parent ma już link do innej gałęzi w drzewie). Prawdą jest jednak, że nawet drzewo sufiksów oparte o prostszą implementacje jest znacząco szybsze od drzewa Trie i może być wystarczająco szybkie dla tekstów średniej długości. Implementację próbowałem naprawić (tj. zobaczyć, że czas wykonania będzie krótszy niż dla prostszej) pół soboty i niestety okazałem się bezsilny wobec owego problemu. 